In [1]:
# remove first if exists
!rm -rf arabic-poetry-speech-classification.git
!git clone https://github.com/MagedSaeed/arabic-poetry-speech-classification.git

Cloning into 'arabic-poetry-speech-classification'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 36 (delta 15), reused 26 (delta 9), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [2]:
# mount the drive to get the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# put the task path you want to work on here
%cd arabic-poetry-speech-classification/signal-classification

/content/arabic-poetry-speech-classification/signal-classification


In [4]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install lang_trans
!pip install arabic_reshaper
!pip install python-bidi
!pip install pydub
!pip install soundfile
!pip install jiwer
!pip install PyArabic

     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 895 kB 54.2 MB/s 
     |████████████████████████████████| 61 kB 469 kB/s 
     |████████████████████████████████| 3.3 MB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 4.2 MB/s 
     |████████████████████████████████| 243 kB 36.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 132 kB 66.5 MB/s 
     |████████████████████████████████| 271 kB 43.7 MB/s 
     |████████████████████████████████| 192 kB 59.7 MB/s 
     |████████████████████████████████| 160 kB 53.5 MB/s 
  Created wheel for lang-trans: filename=lang_trans-0.6.0-py3-none-any.whl size=6345 sha256=361e322aa6482139a4dbf9ed04d7b65ac5a170f22bab91ad911b15f40f5f4fd8
  

In [5]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
import soundfile as sf
import arabic_reshaper
import tensorflow as tf
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
from trainer import CTCTrainer
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from models import Wav2Vec2ClassificationModel
from processors import CustomWav2Vec2Processor
from typing import Any, Dict, List, Optional, Union
from keras.preprocessing.sequence import pad_sequences
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from arg_parsers import DataTrainingArguments, ModelArguments, DataCollatorCTCWithPadding
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
% matplotlib inline

In [6]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/All_poems.zip .

In [7]:
if os.path.exists('dataset'):
  if len(os.listdir('dataset')) == 0:
    os.system('unzip All_poems.zip -d dataset')
else:
  os.system('unzip All_poems.zip -d dataset')

In [8]:
!mkdir -p dataset_wav

In [9]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/testset.csv .

In [10]:
metadata_test_path = 'testset.csv'
dataset_folder = 'dataset'
dataset_wav_folder = 'dataset_wav'

In [11]:
test_metadata = pd.read_csv(metadata_test_path)

In [12]:
sample_rates = set()
for file_path in test_metadata['Utterance name']:
  complete_path = f'{dataset_folder}/{file_path}'
  complete_wav_path = f'{dataset_wav_folder}/{file_path}'
  # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
  audio = AudioSegment.from_file(complete_path)
  sample_rates.add(audio.frame_rate)
  audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
sample_rates

{44100, 48000}

## dataset processing

In [13]:
test_metadata = pd.read_csv(metadata_test_path)
test_dataset = Dataset.from_pandas(test_metadata)

# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

labels = {
    bahr: bahr_index
    for bahr_index, bahr in enumerate(sorted(set(test_metadata["Bahr"])))
}
print("labels are:", labels)
print("len:", len(labels))

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["label"] = labels[batch["Bahr"]]
    return batch

test_dataset = test_dataset.map(
    speech_file_to_array_fn,
    remove_columns=test_dataset.column_names,
    num_proc=1,
)
test_dataset

labels are: {'البسيط': 0, 'الخفيف': 1, 'الرجز': 2, 'الرمل': 3, 'السريع': 4, 'الطويل': 5, 'الكامل': 6, 'المتدارك': 7, 'المتقارب': 8, 'المجتث': 9, 'المديد': 10, 'المضارع': 11, 'المقتضب': 12, 'المنسرح': 13, 'الهزج': 14, 'الوافر': 15}
len: 16


  0%|          | 0/367 [00:00<?, ?ex/s]

Dataset({
    features: ['speech', 'sampling_rate', 'label'],
    num_rows: 367
})

## prepare the model and predict

In [15]:
text_classification_model_dir = "/content/drive/MyDrive/KFUPM-Master/ICS606/Models/arabic_poetry_text_classification_model.h5"
text_classification_model = tf.keras.models.load_model(text_classification_model_dir)

processor = Wav2Vec2Processor.from_pretrained("bakrianoo/sinai-voice-ar-stt")
# model = Wav2Vec2ForCTC.from_pretrained("bakrianoo/sinai-voice-ar-stt").to("cuda").eval()
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600").to("cuda").eval()

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
# from arbml poetry classification model
char2idx = {
  ' ': 1,
  '#': 2,
  'ء': 3,
  'آ': 4,
  'أ': 5,
  'ؤ': 6,
  'إ': 7,
  'ئ': 8,
  'ا': 9,
  'ب': 10,
  'ة': 11,
  'ت': 12,
  'ث': 13,
  'ج': 14,
  'ح': 15,
  'خ': 16,
  'د': 17,
  'ذ': 18,
  'ر': 19,
  'ز': 20,
  'س': 21,
  'ش': 22,
  'ص': 23,
  'ض': 24,
  'ط': 25,
  'ظ': 26,
  'ع': 27,
  'غ': 28,
  'ف': 29,
  'ق': 30,
  'ك': 31,
  'ل': 32,
  'م': 33,
  'ن': 34,
  'ه': 35,
  'و': 36,
  'ى': 37,
  'ي': 38
}
meters_mapping = {0: 4, 1: 6, 2: 8, 3: 7, 4: 13, 5: 10, 6: 9, 7: 3, 8: 0, 9: 1, 10: 5, 11: 15, 12: 14, 13: 2}
# {'البسيط': 0, 'الخفيف': 1, 'الرجز': 2, 'الرمل': 3, 'السريع': 4, 'الطويل': 5, 'الكامل': 6, 'المتدارك': 7, 'المتقارب': 8, 'المجتث': 9, 'المديد': 10, 'المضارع': 11, 'المقتضب': 12, 'المنسرح': 13, 'الهزج': 14, 'الوافر': 15}


In [19]:
def put_hash_after_middle_word(sentence):
  middle_char = len(sentence)//2
  closest_space_index = middle_char
  while True:
    try:
      if sentence[closest_space_index] == ' ':
        break
      else:
        closest_space_index += 1 
    except:
      closest_space_index = len(sentence) - 1
      break
  sentence = sentence[:closest_space_index] + ' #' + sentence[closest_space_index:]
  return sentence

def classify_sentences(sentences):
  classes = list()
  for sentence in sentences:
    sentence = araby.strip_tashkeel(sentence)
    sentence = put_hash_after_middle_word(sentence)
    print(sentence)
    sequence = [char2idx[char] for char in sentence if char2idx.get(char)]
    sequence = pad_sequences([sequence], maxlen = 100, padding='post', value=0)
    preds = text_classification_model.predict(sequence)[0]
    pred = meters_mapping[np.argmax(preds, 0).astype('int')]
    classes.append(pred)
  return classes

def predict(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        predicted = torch.argmax(model(inputs.input_values.to("cuda")).logits, dim=-1)
    predicted[predicted == -100] = processor.tokenizer.pad_token_id  # see fine-tuning script
    batch["predicted"] = processor.batch_decode(predicted)
    batch["pred_label"] = classify_sentences(batch["predicted"])
    return batch

## Test Results

In [20]:
test_dataset_with_pred = test_dataset.map(predict, batched=True, batch_size=16, remove_columns=['speech'])

INFO:tensorflow:Assets written to: ram://a1f40681-b05f-4f60-bfe3-eb4a1bf37991/assets


INFO:tensorflow:Assets written to: ram://a1f40681-b05f-4f60-bfe3-eb4a1bf37991/assets


  0%|          | 0/23 [00:00<?, ?ba/s]

In [21]:
show_label_pred = lambda item: print(item['pred_label'], item['label'])
show_label_pred(test_dataset_with_pred[4])

1 1


In [22]:
labels = [item['label'] for item in test_dataset_with_pred]
pred_labels = [item['pred_label'] for item in test_dataset_with_pred]
print(classification_report(labels,pred_labels))
print(confusion_matrix(labels, pred_labels))

              precision    recall  f1-score   support

           0       0.46      0.86      0.60        14
           1       0.55      0.84      0.67        19
           2       0.12      0.25      0.17         4
           3       0.89      0.82      0.85        38
           4       0.89      0.89      0.89        18
           5       0.94      0.78      0.86       102
           6       0.88      0.70      0.78        20
           7       0.94      0.68      0.79        22
           8       0.47      0.80      0.59        10
           9       0.32      0.88      0.47         8
          10       0.75      0.43      0.55        14
          11       0.00      0.00      0.00        18
          12       0.00      0.00      0.00         2
          13       0.73      0.96      0.83        23
          14       0.58      1.00      0.73        15
          15       0.94      0.72      0.82        40

    accuracy                           0.74       367
   macro avg       0.59   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
